In [1]:
import pandas as pd
import numpy as np
import sshtunnel
import mysql.connector
from datetime import datetime, timedelta
import logging
import os

In [2]:
class NumpyMySQLConverter(mysql.connector.conversion.MySQLConverter):
    """ A mysql.connector Converter that handles Numpy types """

    def _float32_to_mysql(self, value):
        return float(value)

    def _float64_to_mysql(self, value):
        return float(value)

    def _int32_to_mysql(self, value):
        return int(value)

    def _int64_to_mysql(self, value):
        return int(value)

In [3]:
def pandas_query(query):
    with sshtunnel.SSHTunnelForwarder(('207.140.57.162', 101),
            ssh_username='dwilliams', ##'tfenton',
            ssh_password='S0larEngi3200&', ##'!@&sunnyengie',
            remote_bind_address=('localhost', 3306)

    ) as tunnel:
        connection = mysql.connector.connect(user='sdslite', password='1234qw!', ##'tfenton_update', password='1234qw!',
                                  host='192.168.2.204', port='3306',
                                  database='sdsprod', connection_timeout=84000)
        query_results = pd.read_sql_query(query, connection)
        connection.close()
        return query_results


def mysql_insert(query):
    with sshtunnel.SSHTunnelForwarder(('207.140.57.162', 101),
                                      ssh_username='dwilliams', ##'tfenton',
                                      ssh_password='S0larEngi3200&', ##'!@&sunnyengie',
                                      remote_bind_address=('localhost', 3306)

    ) as tunnel:
        connection = mysql.connector.connect(user='sdslite', password='1234qw!', ##'tfenton_update', password='1234qw!',
                                  host='192.168.2.204', port='3306',
                                  database='sdsprod', connection_timeout=84000)
        cursor = connection.cursor()
        cursor.execute('SET sql_mode = ""')
        cursor.execute(query)
        connection.commit()

    cursor.close()
    connection.close()

In [4]:
def add_new_records(new_data):
    #for index, new_data in data_to_add.iterrows():
        #print(new_data)
    data = new_data.replace(np.nan, 0).to_dict()
    #print(data)
    query = ("INSERT into 600_monthly_reporting (month, site_id, site_name, reporting, month_prod, notes, meter_export, meter_import, meter_net, pvsyst_EGrid_lender, pvsyst_EGrid_tax, prod_monthly_lender_ae, prod_monthly_tax_ae, prod_monthly_lender_wa_ae, prod_monthly_tax_wa_ae, uptime, poa_whm2, pvsyst_GlobInc, poa_monthly_ae, cloudy_days, rainy_days, snow_days, snowdepth_min, snowdepth_max, snowdepth_mean, inv_export_sum, inv_count, prod_qtd_lender_ae, prod_qtd_tax_ae, prod_hytd_lender_ae, prod_hytd_tax_ae, prod_ytd_lender_ae, prod_ytd_tax_ae, prod_qtd_lender_wa_ae, prod_qtd_tax_wa_ae, prod_hytd_lender_wa_ae, prod_hytd_tax_wa_ae, prod_ytd_lender_wa_ae, prod_ytd_tax_wa_ae, poa_qtd_ae, poa_hytd_ae, poa_ytd_ae, ghi_whm2, pvsyst_GlobHor, ghi_monthly_ae, ghi_qtd_ae, ghi_hytd_ae, ghi_ytd_ae, ws_poa_whm2, ws_ghi_whm2, meteomatics_poa_whm2, meteomatics_ghi_whm2, meter_record_count, inv_record_avg_count, ws_record_count, meter_day_count, inv_day_count, ws_day_count, meter_first_seen, meter_last_seen, inv_first_seen, inv_last_seen, ws_first_seen, ws_last_seen) VALUES (%(month)s, %(site_id)s, %(site_name)s, %(reporting)s,\
             %(month_prod)s, %(notes)s,\
             %(meter_export)s, %(meter_import)s, %(meter_net)s, %(pvsyst_EGrid_lender)s, %(pvsyst_EGrid_tax)s,\
             %(prod_monthly_lender_a/e)s, %(prod_monthly_tax_a/e)s,\
             %(prod_monthly_lender_wa_a/e)s, %(prod_monthly_tax_wa_a/e)s, %(uptime)s,\
             %(poa_whm2)s, %(pvsyst_GlobInc)s, %(poa_monthly_a/e)s,\
             %(cloudy_days)s, %(rainy_days)s, %(snow_days)s,\
             %(snowdepth_min)s, %(snowdepth_max)s, %(snowdepth_mean)s,\
             %(inv_export_sum)s, %(inv_count)s,\
             %(prod_qtd_lender_a/e)s, %(prod_qtd_tax_a/e)s, %(prod_hytd_lender_a/e)s,\
             %(prod_hytd_tax_a/e)s, %(prod_ytd_lender_a/e)s, %(prod_ytd_tax_a/e)s,\
             %(prod_qtd_lender_wa_a/e)s, %(prod_qtd_tax_wa_a/e)s, %(prod_hytd_lender_wa_a/e)s,\
             %(prod_hytd_tax_wa_a/e)s, %(prod_ytd_lender_wa_a/e)s, %(prod_ytd_tax_wa_a/e)s,\
             %(poa_qtd_a/e)s, %(poa_hytd_a/e)s, %(poa_ytd_a/e)s, %(ghi_whm2)s,\
             %(pvsyst_GlobHor)s, %(ghi_monthly_a/e)s, %(ghi_qtd_a/e)s, %(ghi_hytd_a/e)s, %(ghi_ytd_a/e)s,\
             %(ws_poa_whm2)s, %(ws_ghi_whm2)s, %(meteomatics_poa_whm2)s, %(meteomatics_ghi_whm2)s,\
             %(meter_record_count)s, %(inv_record_avg_count)s, %(ws_record_count)s,\
             %(meter_day_count)s, %(inv_day_count)s, %(ws_day_count)s, %(meter_first_seen)s, %(meter_last_seen)s,\
             %(inv_first_seen)s, %(inv_last_seen)s, %(ws_first_seen)s, %(ws_last_seen)s)")
    #print(query % data)
    mysql_insert(query, data)


In [5]:
def update_records(new_data):
    #for index, new_data in mid_month_data.iterrows():
    data = new_data.replace(np.nan, 0).astype(str).to_dict()
    query = ("UPDATE 600_monthly_reporting SET site_name = %(site_name)s\
             , meter_export = %(meter_export)s, meter_import = %(meter_import)s, meter_net = %(meter_net)s\
             , pvsyst_EGrid_lender = %(pvsyst_EGrid_lender)s, pvsyst_EGrid_tax = %(pvsyst_EGrid_tax)s\
             , prod_monthly_lender_ae = %(prod_monthly_lender_a/e)s, prod_monthly_tax_ae = %(prod_monthly_tax_a/e)s\
             , prod_qtd_lender_ae = %(prod_qtd_lender_a/e)s, prod_qtd_tax_ae = %(prod_qtd_tax_a/e)s\
             , prod_hytd_lender_ae = %(prod_hytd_lender_a/e)s, prod_hytd_tax_ae = %(prod_hytd_tax_a/e)s\
             , prod_ytd_lender_ae = %(prod_ytd_lender_a/e)s, prod_ytd_tax_ae = %(prod_ytd_tax_a/e)s\
             , prod_monthly_lender_wa_ae = %(prod_monthly_lender_wa_a/e)s, prod_monthly_tax_wa_ae = %(prod_monthly_tax_wa_a/e)s\
             , prod_qtd_lender_wa_ae = %(prod_qtd_lender_wa_a/e)s, prod_qtd_tax_wa_ae = %(prod_qtd_tax_wa_a/e)s\
             , prod_hytd_lender_wa_ae = %(prod_hytd_lender_wa_a/e)s, prod_hytd_tax_wa_ae = %(prod_hytd_tax_wa_a/e)s\
             , prod_ytd_lender_wa_ae = %(prod_ytd_lender_wa_a/e)s, prod_ytd_tax_wa_ae = %(prod_ytd_tax_wa_a/e)s\
             , inv_export_sum = %(inv_export_sum)s, inv_count = %(inv_count)s\
             , pvsyst_GlobInc = %(pvsyst_GlobInc)s, poa_whm2 = %(poa_whm2)s, poa_monthly_ae = %(poa_monthly_a/e)s\
             , poa_qtd_ae = %(poa_qtd_a/e)s, poa_hytd_ae = %(poa_hytd_a/e)s\
             , poa_ytd_ae = %(poa_ytd_a/e)s, ghi_whm2 = %(ghi_whm2)s, pvsyst_GlobHor = %(pvsyst_GlobHor)s\
             , ghi_monthly_ae = %(ghi_monthly_a/e)s, month_prod = %(month_prod)s\
             , ghi_qtd_ae = %(ghi_qtd_a/e)s, ghi_hytd_ae = %(ghi_hytd_a/e)s\
             , ghi_ytd_ae = %(ghi_ytd_a/e)s, notes = %(notes)s\
             , cloudy_days = %(cloudy_days)s, rainy_days = %(rainy_days)s, snow_days = %(snow_days)s\
             , snowdepth_min = %(snowdepth_min)s, snowdepth_max = %(snowdepth_max)s, snowdepth_mean = %(snowdepth_mean)s\
             , ws_poa_whm2 = %(ws_poa_whm2)s, ws_ghi_whm2 = %(ws_ghi_whm2)s\
             , meteomatics_poa_whm2 = %(meteomatics_poa_whm2)s, meteomatics_ghi_whm2 = %(meteomatics_ghi_whm2)s\
             , meter_day_count = %(meter_day_count)s, reporting = %(reporting)s\
             , inv_day_count = %(inv_day_count)s, ws_day_count = %(ws_day_count)s\
             , uptime = %(uptime)s, meter_record_count = %(meter_record_count)s\
             , inv_record_avg_count = %(inv_record_avg_count)s, ws_record_count = %(ws_record_count)s\
             , meter_first_seen = %(meter_first_seen)s, meter_last_seen = %(meter_last_seen)s\
             , inv_first_seen = %(inv_first_seen)s, inv_last_seen = %(inv_last_seen)s\
             , ws_first_seen = %(ws_first_seen)s, ws_last_seen = %(ws_last_seen)s\
             WHERE site_id = %(site_id)s AND month= %(month)s")
    #print(query % data)
    mysql_insert(query, data)

In [6]:
def redundant_meters(meters_601, primary_meter1_id, red_meter1_id, primary_meter2_id=None, red_meter2_id=None, red_meter3_id=None):
    if primary_meter2_id is None:
        if red_meter2_id is None:
            if meters_601[meters_601['device_id'] == primary_meter1_id]['export_energy'].iloc[0] <= 0:
                if meters_601[meters_601['device_id'] == red_meter1_id]['export_energy'].iloc[0] > 0:
                    meters_601 = meters_601[meters_601['device_id'] != primary_meter1_id]
                else:
                    meters_601 = meters_601[meters_601['device_id'] != red_meter1_id]
            else:
                meters_601 = meters_601[meters_601['device_id'] != red_meter1_id]
        elif red_meter3_id is None:
            if meters_601[meters_601['device_id'] == primary_meter1_id]['export_energy'].iloc[0] <= 0:
                if meters_601[(meters_601['device_id'] == red_meter1_id) & (meters_601['device_id'] == red_meter2_id)][
                                'export_energy'].sum() > 0:
                    meters_601 = meters_601[meters_601['device_id'] != primary_meter1_id]
                else:
                    meters_601 = meters_601[(meters_601['device_id'] != red_meter1_id) & (meters_601['device_id'] != red_meter2_id)]
            else:
                meters_601 = meters_601[(meters_601['device_id'] != red_meter1_id) & (meters_601['device_id'] != red_meter2_id)]
        else:
            if meters_601[meters_601['device_id'] == primary_meter1_id]['export_energy'].iloc[0] <= 0:
                if meters_601[(meters_601['device_id'] == red_meter1_id) & (meters_601['device_id'] == red_meter2_id) & (meters_601['device_id'] == red_meter3_id)][
                                'export_energy'].sum() > 0:
                    meters_601 = meters_601[meters_601['device_id'] != primary_meter1_id]
                else:
                    meters_601 = meters_601[(meters_601['device_id'] != red_meter1_id) & (meters_601['device_id'] != red_meter2_id) & (
                                meters_601['device_id'] != red_meter3_id)]
            else:
                meters_601 = meters_601[(meters_601['device_id'] != red_meter1_id) & (meters_601['device_id'] != red_meter2_id) & (
                                meters_601['device_id'] != red_meter3_id)]
    else:
        if red_meter2_id is None:
            if meters_601[(meters_601['device_id'] == primary_meter1_id) & (meters_601['device_id'] == primary_meter2_id)][
                    'export_energy'].sum() > 0:
                if meters_601[meters_601['device_id'] == red_meter1_id]['export_energy'].iloc[0] > 0:
                    meters_601 = meters_601[(meters_601['device_id'] != primary_meter1_id) & (meters_601['device_id'] != primary_meter2_id)]
                else:
                    meters_601 = meters_601[meters_601['device_id'] != red_meter1_id]
            else:
                meters_601 = meters_601[meters_601['device_id'] != red_meter1_id]
        elif red_meter3_id is None:
            if meters_601[(meters_601['device_id'] == primary_meter1_id) & (meters_601['device_id'] == primary_meter2_id)][
                    'export_energy'].sum() > 0:
                if meters_601[(meters_601['device_id'] == red_meter1_id) & (meters_601['device_id'] == red_meter2_id)][
                                'export_energy'].sum() > 0:
                    meters_601 = meters_601[
                        (meters_601['device_id'] != primary_meter1_id) & (meters_601['device_id'] != primary_meter2_id)]
                else:
                    meters_601 = meters_601[(meters_601['device_id'] != red_meter1_id) & (meters_601['device_id'] != red_meter2_id)]
            else:
                meters_601 = meters_601[(meters_601['device_id'] != red_meter1_id) & (meters_601['device_id'] != red_meter2_id)]
        else: # if there are 3 redud meters
            if meters_601[(meters_601['device_id'] == primary_meter1_id) & (meters_601['device_id'] == primary_meter2_id)][
                    'export_energy'].sum() > 0:
                if meters_601[(meters_601['device_id'] == red_meter1_id) & (meters_601['device_id'] == red_meter2_id) & (meters_601['device_id'] == red_meter3_id)][
                                'export_energy'].sum() > 0:
                    meters_601 = meters_601[
                        (meters_601['device_id'] != primary_meter1_id) & (meters_601['device_id'] != primary_meter2_id)]
                else:
                    meters_601 = meters_601[(meters_601['device_id'] != red_meter1_id) & (meters_601['device_id'] != red_meter2_id) & (
                                meters_601['device_id'] != red_meter3_id)]
            else:
                meters_601 = meters_601[(meters_601['device_id'] != red_meter1_id) & (meters_601['device_id'] != red_meter2_id) & (
                                meters_601['device_id'] != red_meter3_id)]
    return meters_601

In [7]:
def preprocess_site_data(meters_601, meters_002, sites_002):
    # make only ion meters show up for xcel sites, unless the ion meter is broken
    xcel_sites = meters_601[meters_601['site_name'].str.lower().str.contains("xcel")].drop_duplicates(subset=['site_name'])
    for idx, xcel_site in xcel_sites.iterrows():
        try:
            xcel_site_data = meters_601[meters_601['site_id'] == xcel_site['site_id']]
            xcel_site_data.index = xcel_site_data['device_id']
            #print(xcel_site_data[['export_energy','oem']])
            for idx, site in xcel_site_data.iterrows():
                xcel_site_data.loc[site['device_id'], 'oem'] = meters_002[meters_002['id'] == site['device_id']]['oem'].iloc[0]
            #print(xcel_site_data[xcel_site_data['export_signal'] == 'Delivered Energy (kWh)']['export_energy'].iloc[0])
            # check if ion meters have data
            if xcel_site_data[xcel_site_data['oem'] == 'Ion']['export_energy'].iloc[0] == 0:
                if xcel_site_data[xcel_site_data['oem'] == 'Elkor']['export_energy'].iloc[0] != 0:
                    # if ion meter doesn't have data and elkor does, use elkor in table 600
                    meters_601 = meters_601[~meters_601['device_id'] == xcel_site_data[xcel_site_data['oem'] == 'ion']['device_id'].iloc[0]]
            else: # if ion has data, drop elkor
                #print(xcel_site_data[xcel_site_data['oem'] != 'ion'][
                #        'device_id'].iloc[0])
                meters_601 = meters_601[
                    meters_601['device_id'] != xcel_site_data[xcel_site_data['oem'] != 'ion'][
                        'device_id'].iloc[0]]
        except:
            logging.exception("traceback")
            print("xcel site " + xcel_site['site_name'] + " failed meter check")
    # natick has a redundant meter
    meters_601 = redundant_meters(meters_601, 101965, 101942)
    # Athens A has a redundant meter
    meters_601 = redundant_meters(meters_601, 105991, 105994)
    # Athens B has redundant meters
    meters_601 = redundant_meters(meters_601, 106025, 106024, red_meter2_id=106033, red_meter3_id=106051)
    # Anoka has redundant meters
    meters_601 = redundant_meters(meters_601, 105932, 105934, red_meter2_id=105939)
    # Richmond A
    return meters_601, sites_002

In [8]:
def primary_meter_check(site,site_meter, meters_601):
    site_meter_primary = pd.DataFrame()
    site_meter_secondary = pd.DataFrame()
    secondary_meter_sum = 0
    primary_meter_sum = 0
    mon_prod = 0
    for idx, meter in site_meter.iterrows():
        if meter['net_energy'] > 0:
            mon_prod = meter['net_energy']
        elif meter['export_energy'] > 0:
            mon_prod = meter['export_energy']
        if mon_prod > 0:
            if str(meter['device_id']) in str(site['primary_meters']):
                primary_meter_sum = primary_meter_sum + mon_prod
                site_meter = meters_601[meters_601['device_id'] == meter['device_id']]
                site_meter_primary = pd.concat([site_meter_primary,site_meter],axis=0)
            else:
                secondary_meter_sum = secondary_meter_sum + mon_prod
                site_meter_secondary = pd.concat([site_meter_secondary, meter], axis=0)
            if primary_meter_sum > 0:
                #if the meters are within 5% of each other then use primary
                if ((secondary_meter_sum - primary_meter_sum) / primary_meter_sum) < 0.05:
                    site_meter = site_meter_primary
                elif secondary_meter_sum > primary_meter_sum:
                    site_meter = site_meter_secondary
            else:
                site_meter = secondary_meter_sum
    return site_meter

In [9]:
def update_meter_uptime(new_data):
    #for index, new_data in mid_month_data.iterrows():
    #for index, value in new_data.iteritems():
    #    try:
    #        if new_data[index].dtype == np.int64:
    #            new_data[index] = float(new_data[index])
    #    except:
    #        new_data = new_data
    data = new_data.replace(np.nan, 0).to_dict()
    query = ("UPDATE 601_meters_monthly SET uptime =  %(uptime)s WHERE device_id= %(device_id)s AND month= %(month)s")
    mysql_insert(query, data)

In [10]:
def update_inv_uptime(new_data):
    #for index, new_data in mid_month_data.iterrows():
    #for index, value in new_data.iteritems():
    #    try:
    #        if new_data[index].dtype == np.int64:
    #            new_data[index] = float(new_data[index])
    #    except:
    #        new_data = new_data
    data = new_data.replace(np.nan, 0).to_dict()
    query = ("UPDATE 602_inverters_monthly SET uptime= %(uptime)s WHERE device_id= %(device_id)s AND month= %(month)s")
    #print(query % data)
    mysql_insert(query, data)

In [11]:
def uptime(new_table_600, days_ago, site, site_ws, site_inverters, site_meter, all_devices):
    inv_uptime_list = []
    meteo_query = "select site_name, site_id, UTC, poa_isotropic_W, snowdepth_cm from sdsprod.weather_meteomatics where site_id = %s \
                            and UTC >= date_format((curdate() - interval %s day),'%s') \
                            and UTC <= date_format(curdate()-interval %s day,'%s')" % (
        site_ws['site_id'].iloc[0], days_ago, '%Y-%m-01 00:00:00', days_ago, '%Y-%m-%d 23:59:59')
    meteo_events = pandas_query(meteo_query)
    meteo_poa_events = pd.DataFrame()
    meteo_poa_events['poa_wm2'] = meteo_events['poa_isotropic_W']
    meteo_poa_events.index = pd.to_datetime(meteo_events['UTC'])
    meteo_poa_events.index = meteo_poa_events.index.tz_localize(tz='UTC')
    meteo_poa_events = meteo_poa_events['poa_wm2'].resample("H").mean()
    # now snow
    meteo_snow_events = pd.DataFrame()
    meteo_snow_events['snowdepth_cm'] = meteo_events['snowdepth_cm'].astype(float)
    meteo_snow_events.index = pd.to_datetime(meteo_events['UTC'])
    meteo_snow_events.index = meteo_snow_events.index.tz_localize(tz='UTC')
    meteo_snow_events = meteo_snow_events['snowdepth_cm'].resample("H").mean()
    if len(site_ws['poa_signal'].iloc[0]) > 3:
        ws_device_id = site_ws['device_id'].iloc[0].split('_')[0]
        ws_002 = all_devices[all_devices['id'] == int(ws_device_id)]
        if len(ws_device_id) > 6:
            ws_device_id = site_ws['device_id'].iloc[0].split('_')[0][:-1]
        #print(ws_device_id)
        poa_query = "select * from sdsprod.events where device_id = %s and type in ('%s') \
                        and clock >= date_format((curdate() - interval %s day), '%s') \
                        and clock <= date_format(curdate()-interval %s day, '%s')" % (
            ws_device_id, site_ws['poa_signal'].iloc[0], days_ago, '%Y-%m-01 00:00:00', days_ago,
            '%Y-%m-%d 23:59:59')
        poa_events = pandas_query(poa_query)
        # print(poa_events.head(), "poa_events!")
        poa_events['poa_wm2'] = poa_events['value']
        poa_events.index = pd.to_datetime(poa_events['clock'])
        poa_events.index = poa_events.index.tz_localize(tz=ws_002['data_tz'].iloc[0])
        poa_events = poa_events['poa_wm2'].resample("H").mean()
        poa_events.index = poa_events.index.tz_convert(tz='UTC')
    else:
        poa_events = meteo_poa_events

    meter_uptime_list = []
    for idx, meter in site_meter.iterrows():
        meter_002 = all_devices[all_devices['id'] == int(meter['device_id'])]
        if len(meter['net_signal']) > 3:
            energy_signal = meter['net_signal']
        elif len(meter['export_signal']) > 3:
            energy_signal = meter['export_signal']
        else:
            energy_signal = 0
            #print(site['site_name'], meter['device_id'], "no meter energy for month")
        if not energy_signal == 0:
            meter_query = "select * from sdsprod.events where device_id = %s and type = '%s' \
                                and clock >= date_format((curdate() - interval %s day),'%s') \
                                and clock <= date_format(curdate()-interval %s day,'%s')" % \
                          (meter['device_id'], energy_signal, days_ago, '%Y-%m-01 00:00:00', days_ago,
                           '%Y-%m-%d 23:59:59')
            meter_events = pandas_query(meter_query)
            meter_events['meter_energy'] = pd.to_numeric(meter_events['value'])
            meter_events['meter_power'] = meter_events['meter_energy'].diff()
            meter_events.index = pd.to_datetime(meter_events['clock'])
            try:
                meter_events.index = meter_events.index.tz_localize(tz=meter_002['data_tz'].iloc[0])
            except:
                meter_events.index = meter_events.index.tz_localize(tz='UTC')
            meter_energy = meter_events['meter_energy'].resample("H").max()
            meter_energy.index = meter_energy.index.tz_convert(tz='UTC')
            meter_power = meter_events['meter_power'].resample("H").max()
            meter_power.index = meter_power.index.tz_convert(tz='UTC')
            # Uptime calc
            full_uptime_df = pd.concat([poa_events, meter_power, meter_energy, meteo_snow_events], axis=1)
            full_uptime_df.columns = ['poa_wm2', 'meter_power', 'meter_energy', 'snowdepth_cm']
            full_uptime_df = full_uptime_df.replace(to_replace=np.nan, value=0)
            # print(full_uptime_df)
            uptime_intervals = full_uptime_df[full_uptime_df['poa_wm2'] > site['uptime_thresh']]
            meter_uptime = (
                        uptime_intervals[(uptime_intervals['meter_power'] > 0) | (uptime_intervals['meter_energy'] == 0)
                                         | (uptime_intervals['snowdepth_cm'] > 0)]['meter_energy'].count()
                        / uptime_intervals['meter_energy'].count())
            meter['uptime'] = str(meter_uptime)
            meter_uptime_list.append(meter_uptime)
            update_meter_uptime(meter.astype(str))
    inv_count = len(site_inverters)
    full_inv_events = pd.DataFrame()
    if inv_count > 0:
        for idx, inv in site_inverters.iterrows():
            inv_002 = all_devices[all_devices['id'] == int(inv['device_id'])]
            if len(inv['net_signal']) > 3:
                energy_signal = inv['net_signal']
            elif len(inv['export_signal']) > 3:
                energy_signal = inv['export_signal']
            else:
                energy_signal = 0
                #print(site['site_name'], inv['device_id'], "no inv energy for month")
            if not energy_signal == 0:
                inv_query = "select * from sdsprod.events where device_id = %s and type = '%s' \
                                and clock >= date_format((curdate() - interval %s day),'%s') \
                                and clock <= date_format(curdate()-interval %s day,'%s')" % \
                            (inv['device_id'], energy_signal, days_ago, '%Y-%m-01 00:00:00', days_ago,
                             '%Y-%m-%d 23:59:59')
                inv_events = pandas_query(inv_query)
                inv_events['inv_energy'] = pd.to_numeric(inv_events['value'])
                inv_events['inv_power'] = inv_events['inv_energy'].diff()
                inv_events.index = pd.to_datetime(inv_events['clock'])
                try:
                    inv_events.index = inv_events.index.tz_localize(tz=inv_002['data_tz'].iloc[0])
                except:
                    inv_events.index = inv_events.index.tz_localize(tz='UTC')
                inv_power = inv_events['inv_power'].resample("H").max()
                inv_power.index = inv_power.index.tz_convert(tz='UTC')
                inv_energy = inv_events['inv_energy'].resample("H").max()
                inv_energy.index = inv_energy.index.tz_convert(tz='UTC')
                # print(inv_events.head())
                # Uptime calc
                full_uptime_df = pd.concat([poa_events, inv_power, inv_energy, meter_power, meteo_snow_events], axis=1)
                full_uptime_df.columns = ['poa_wm2', 'inv_power', 'inv_energy', 'meter_energy', 'snowdepth_cm']
                full_uptime_df = full_uptime_df.replace(to_replace=np.nan, value=0)
                # print(full_uptime_df)
                uptime_intervals = full_uptime_df[full_uptime_df['poa_wm2'] > site['uptime_thresh']]
                inv_uptime = (
                            uptime_intervals[(uptime_intervals['inv_power'] > 0) | (uptime_intervals['inv_energy'] == 0)
                                             | (uptime_intervals['snowdepth_cm'] > 0)]['inv_power'].count()
                            / uptime_intervals['inv_power'].count())
                inv['uptime'] = str(inv_uptime)
                inv['month'] = str(inv['month'])
                inv_uptime_list.append(inv_uptime)
                update_inv_uptime(inv.astype(str))
                # full_inv_events = pd.concat([full_inv_events, inv_events],axis=1)
        # print(sum(inv_uptime_list) / len(inv_uptime_list))
        # full_events = pd.concat([poa_events, full_inv_events, meter_events, meteo_snow_events], axis=1)
        # full_events.to_csv("/media/engieshare/O&M/Ty/Tasks/Monthly_Reporting/uptime_check.csv")
    if not len(inv_uptime_list) == 0:
        if not str(sum(inv_uptime_list) / len(inv_uptime_list)) == "nan":
            new_table_600.loc[site['id'], 'uptime'] = (sum(inv_uptime_list) / len(inv_uptime_list))
        else:
            new_table_600.loc[site['id'], 'uptime'] = "n/a"
    elif not len(meter_uptime_list) == 0:
        if not str(sum(meter_uptime_list) / len(meter_uptime_list)) == "nan":
            new_table_600.loc[site['id'], 'uptime'] = (sum(meter_uptime_list) / len(meter_uptime_list))
        else:
            new_table_600.loc[site['id'], 'uptime'] = "n/a"
    else:
        new_table_600.loc[site['id'], 'uptime'] = "n/a"
    # print(site['site_name'], new_table_600.loc[site['id'], 'uptime'])
    return new_table_600


In [ ]:
def main():
    os.chdir("/")
    logging.basicConfig(filename="S:/CHI1/Company Data/O&M/Diamond_Williams/Reporting/Codes/monthly_reporting_600.log", filemode='w', level=logging.ERROR)
    #days_ago_list = [450, 422, 391, 361, 330, 300, 269, 238, 208, 177, 147, 116, 85, 57, 26, 1]
    days_ago_list = [1]
    for days_ago in days_ago_list:
        print(days_ago)
        print(str(datetime.strftime(datetime.today() - timedelta(days=days_ago), '%Y-%m')))
        #days_ago = 23
        ## Pull in the current data from the sds
        # all devices in fleet
        all_devices_query = "select id, signal_02, site_id, site_name, first_seen, last_seen, oem, data_tz from sdsprod.002_devices_new"
        all_devices = pandas_query(all_devices_query)
        sites_002 = all_devices.drop_duplicates(subset=['site_id'])
        meter_query = "select id, signal_02, site_id, site_name, dev_ac, oem from sdsprod.002_devices_new where device_type " \
                      "like 'meter'"
        # and last_seen >= (curdate() - interval %s day)
        meters_002 = pandas_query(meter_query)
        sites_006 = pandas_query("select * from 006_siteID_new where id >= 100")
        #monthly data
        meters_601 = pandas_query("select * from 601_meters_monthly where month like date_format(curdate()-interval %s day,'%s')" % (days_ago, '%Y-%m'))
        inverters_602 = pandas_query("select * from 602_inverters_monthly where month like date_format(curdate()-interval %s day,'%s')" % (days_ago, '%Y-%m'))
        ws_603 = pandas_query("select * from 603_weather_stations_monthly where month like date_format(curdate()-interval %s day, '%s')" % (days_ago, '%Y-%m'))
        #pvsyst data
        pvsyst_estimates = pandas_query("select * from pvsyst_production where category not like 'archive'")
        old_table_600 = pandas_query("select * from 600_monthly_reporting")
        new_table_600 = pd.DataFrame()
        #meters_601, sites_002 = preprocess_site_data(meters_601, meters_002, sites_002)
        for idx, site in sites_006.iterrows():
            site_name = site['site_name']
            site_id = site['id']
            site_meter = meters_601[meters_601['site_id'] == site['id']]
            site_inverters = inverters_602[inverters_602['site_id'] == site['id']]
            #site_name_clean = site['site_name'].replace('-CSG', '').split(',')[0]  # to match ws on csg or sites
            site_ws = ws_603[ws_603['site_id'] == site['id']]
            if site_meter.empty:
                site_meter = meters_601[meters_601['group_id'] == site['group_id']]
            if site_inverters.empty:
                site_inverters = inverters_602[inverters_602['group_id'] == site['group_id']]
            if site_ws.empty:
                site_ws = ws_603[ws_603['group_id'] == site['group_id']]
            if not site['primary_meters'] is None:
                site_meter = primary_meter_check(site,site_meter, meters_601)
            site_old_600 = old_table_600[old_table_600['site_id'] == site['id']]
            site_006 = sites_006[sites_006['id'] == site['id']]
            try:
                site_pvsyst = pvsyst_estimates[pvsyst_estimates['site_id'] == site['id']]
                curr_month = float(datetime.strftime(datetime.today() - timedelta(days=days_ago), '%m'))
                curr_year = float(datetime.strftime(datetime.today() - timedelta(days=days_ago), '%Y'))
                site_pvsyst_month = site_pvsyst[site_pvsyst['month'] == curr_month]
                site_pvsyst_months = site_pvsyst[site_pvsyst['month'] > 0]
                site_pvsyst_ytd = site_pvsyst_months[site_pvsyst_months['month'] <= curr_month]
            except:
                curr_month = float(datetime.strftime(datetime.today() - timedelta(days=days_ago), '%m'))
                site_pvsyst_month = pd.DataFrame(0)
                site_pvsyst_months = pd.DataFrame(0)
                site_pvsyst_ytd = pd.DataFrame(0)
            if not site_old_600.empty:
                site_old_600['month_num'] = pd.to_numeric(site_old_600['month'].str.split("-", expand=True)[1], errors='coerse')
                site_old_600['year'] = pd.to_numeric(site_old_600['month'].str.split("-", expand=True)[0],
                                                          errors='coerse')
                site_old_600_ytd = site_old_600[(site_old_600['month_num'] < curr_month) & (site_old_600['year'] == curr_year)]
                #print(site_old_600['poa_whm2'], site_pvsyst_months['GlobInc'])
                if curr_month <= 3:
                    site_pvsyst_qtd = site_pvsyst_months[site_pvsyst_months['month'] <= curr_month]
                    site_old_600_qtd = site_old_600[(site_old_600['month_num'] < curr_month) & (site_old_600['year'] == curr_year)]
                elif curr_month >= 4 and curr_month <= 6:
                    site_pvsyst_quarter_a = site_pvsyst_months[site_pvsyst_months['month'] >= 4]
                    site_pvsyst_quarter = site_pvsyst_quarter_a[site_pvsyst_quarter_a['month'] <= 6]
                    site_pvsyst_qtd = site_pvsyst_quarter_a[site_pvsyst_quarter_a['month'] <= curr_month]
                    site_old_600_quarter_a = site_old_600[(site_old_600['month_num'] >= 4) & (site_old_600['year'] == curr_year)]
                    site_old_600_qtd = site_old_600_quarter_a[site_old_600_quarter_a['month_num'] < curr_month]
                elif curr_month >= 7 and curr_month <= 9:
                    site_pvsyst_quarter_a = site_pvsyst_months[site_pvsyst_months['month'] >= 7]
                    site_pvsyst_qtd = site_pvsyst_quarter_a[site_pvsyst_quarter_a['month'] <= curr_month]
                    site_old_600_quarter_a = site_old_600[(site_old_600['month_num'] >= 7) & (site_old_600['year'] == curr_year)]
                    site_old_600_qtd = site_old_600_quarter_a[site_old_600_quarter_a['month_num'] < curr_month]
                elif curr_month >= 10:
                    site_pvsyst_quarter = site_pvsyst_months[site_pvsyst_months['month'] >= 10]
                    site_pvsyst_qtd = site_pvsyst_quarter[site_pvsyst_quarter['month'] <= curr_month]
                    site_old_600_quarter_a = site_old_600[(site_old_600['month_num'] >= 10) & (site_old_600['year'] == curr_year)]
                    site_old_600_qtd = site_old_600_quarter_a[site_old_600_quarter_a['month_num'] < curr_month]
                if curr_month <= 6:
                    site_pvsyst_hytd = site_pvsyst_months[site_pvsyst_months['month'] <= curr_month]
                    site_old_600_half_year_a = site_old_600[(site_old_600['month_num'] <= 6) & (site_old_600['year'] == curr_year)]
                    site_old_600_hytd = site_old_600_half_year_a[site_old_600_half_year_a['month_num'] < curr_month]
                else:
                    site_pvsyst_half_year = site_pvsyst_months[site_pvsyst_months['month'] > 6]
                    site_pvsyst_hytd = site_pvsyst_half_year[site_pvsyst_half_year['month'] <= curr_month]
                    site_old_600_half_year_a = site_old_600[(site_old_600['month_num'] > 6) & (site_old_600['year'] == curr_year)]
                    site_old_600_hytd = site_old_600_half_year_a[site_old_600_half_year_a['month_num'] < curr_month]
            else:
                #site_pvsyst_quarter = 0
                site_pvsyst_qtd = 0
                #site_old_600_quarter = 0
                site_old_600_qtd = 0
                #site_pvsyst_half_year = 0
                site_pvsyst_hytd = 0
                #site_old_600_half_year = 0
                site_old_600_hytd = 0
                site_pvsyst_ytd = 0
                site_old_600_ytd = 0
            # build the table
            # meters
            new_table_600.loc[site_id, 'site_id'] = site['id']
            new_table_600.loc[site_id, 'site_name'] = site['site_name']
            new_table_600.loc[site_id, 'month'] = str(datetime.strftime(datetime.today() - timedelta(days=days_ago), '%Y-%m'))
            new_table_600.loc[site_id, 'reporting'] = 'y'
            new_table_600.loc[site_id, 'notes'] = ''
            try:
                new_table_600.loc[site_id, 'meter_export'] = site_meter['export_energy'].sum()
                new_table_600.loc[site_id, 'meter_import'] = site_meter['import_energy'].sum()
                new_table_600.loc[site_id, 'meter_net'] = site_meter['net_energy'].sum()
                new_table_600.loc[site_id, 'meter_record_count'] = site_meter['count'].sum()
                ## PVSyst Egrid calculated based on phases
                # For Phase I, degradation should be 0.75%.
                # For Phase II, for lenders we need 0.7% degradation.  The degradation is calculated every 12 months and it is not compounded (year 1 is 100%, year 2 is 100% - 0.70%, year 3 is 100% - (0.70% * 2), year 4 is 100% - (0.70% *3), etc).  For tax equity, degradation should be 0.50%, but it is not compounded (i.e. year 2 is 99.50%, year 3 is at 99.00%, year 4 is at 98.5%,etc)
                # For Phase III, as I noted in my e-mail to you guys, there are two sets of degradation assumptions:
                    # 1.	For lenders, we need to use 0.5% degradation.  The degradation is calculated every 12 months, and is compounded once a year (i.e. production for January of 2019 = January 2018 production * 99.50%, and production for February 2019 = February 2018 production *99.50%).
                    # 2.	For tax equity, we need to use 0.75% degradation, compounded annually, once.
                # For Phase IV, we need to use 0.5%, compounded once a year.
                E_Grid = site_pvsyst_month['E_Grid'].sum()
                first_seen = pd.to_datetime(site_meter['first_seen'].iloc[0])
                years = int(((datetime.today() - first_seen).days / 365))
                site_lender_deg = float(site_006['degradation_rate_lender'].iloc[0]) / 100
                site_tax_deg = float(site_006['degradation_rate_tax'].iloc[0]) / 100
                if years >= 1:
                    if site_006['degradation_compound_term_lender'] == 'annual':
                        new_table_600.loc[site_id, 'pvsyst_EGrid_lender'] = (E_Grid * (1 - (site_lender_deg*years)))
                    else:
                        new_table_600.loc[site_id, 'pvsyst_EGrid_lender'] = E_Grid - ((E_Grid * site_lender_deg) * years)
                    if site_006['degradation_compound_term_tax'] == 'annual':
                        new_table_600.loc[site_id, 'pvsyst_EGrid_tax'] = (E_Grid * (1 - (site_tax_deg*years)))
                    else:
                        new_table_600.loc[site_id, 'pvsyst_EGrid_tax'] = E_Grid - ((E_Grid * site_tax_deg) * years)
                else:
                    new_table_600.loc[site_id, 'pvsyst_EGrid_lender'] = E_Grid
                    new_table_600.loc[site_id, 'pvsyst_EGrid_tax'] = E_Grid
                if (type(site_meter['first_seen'].sum()) == str and type(site_meter['last_seen'].sum()) == str):
                    new_table_600.loc[site_id, 'meter_first_seen'] = site_meter['first_seen'].sum()
                    new_table_600.loc[site_id, 'meter_last_seen'] = site_meter['last_seen'].sum()
                else:
                    new_table_600.loc[site_id, 'meter_first_seen'] = '1990-01-01'
                    new_table_600.loc[site_id, 'meter_last_seen'] = '1990-01-01'
                # transfer the reporting column from 601
                # fm isn't bulit in yet, but one day might be
                if len(site_meter[site_meter['reporting'].str.contains('fm')]) > 0:
                    new_table_600.loc[site_id, 'reporting'] = 'fm'
                if len(site_meter[site_meter['reporting'].str.contains('pd')]) > 0:
                    new_table_600.loc[site_id, 'reporting'] = 'pd'
                elif len(site_meter[site_meter['reporting'].str.contains('nd')]) > 0 and len(
                        site_meter[site_meter['reporting'].str.contains('y')]) > 0:
                    new_table_600.loc[site_id, 'reporting'] = 'pd'
                elif len(site_meter[site_meter['reporting'].str.contains('nd')]) > 0:
                    new_table_600.loc[site_id, 'reporting'] = 'nd'
                else:
                    new_table_600.loc[site_id, 'reporting'] = 'y'
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'meter_export'] = 0
                new_table_600.loc[site_id, 'meter_import'] = 0
                new_table_600.loc[site_id, 'meter_net'] = 0
                new_table_600.loc[site_id, 'meter_record_count'] = 0
                new_table_600.loc[site_id, 'meter_day_count'] = 0
                new_table_600.loc[site_id, 'pvsyst_EGrid_lender'] = 0
                new_table_600.loc[site_id, 'pvsyst_EGrid_tax'] = 0
                new_table_600.loc[site_id, 'meter_first_seen'] = 0
                new_table_600.loc[site_id, 'meter_last_seen'] = 0
                new_table_600.loc[site_id, 'reporting'] = 'nd'

            try:
                new_table_600.loc[site_id, 'meter_day_count'] = site_meter['day_count'].mean()
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'meter_day_count'] = 0
            #calc various a/e
            try:
                if not site_meter['net_energy'].sum() == 0:
                    site_energy_sum = site_meter['net_energy'].sum()
                elif not site_meter['export_energy'].sum() == 0:
                    site_energy_sum = site_meter['export_energy'].sum()
                if site_energy_sum > 0:
                    new_table_600.loc[site_id, 'prod_monthly_lender_a/e'] = (site_energy_sum /
                                                                               new_table_600.loc[site_id, 'pvsyst_EGrid_tax'].sum())
                    new_table_600.loc[site_id, 'prod_monthly_tax_a/e'] = (site_energy_sum /
                                                                            new_table_600.loc[site_id, 'pvsyst_EGrid_tax'].sum())
                else:
                    new_table_600.loc[site_id, 'prod_monthly_lender_a/e'] = 0
                    new_table_600.loc[site_id, 'prod_monthly_tax_a/e'] = 0
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'prod_monthly_lender_a/e'] = 0
                new_table_600.loc[site_id, 'prod_monthly_tax_a/e'] = 0
            try:
                if not site_old_600_ytd['meter_net'].sum() + site_meter['net_energy'].sum() == 0:
                    site_ytd_energy_sum = site_old_600_ytd['meter_net'].sum() + site_meter['net_energy'].sum()
                elif not site_old_600_ytd['meter_export'].sum() + site_meter['export_energy'].sum() == 0:
                    site_ytd_energy_sum = site_old_600_ytd['meter_export'].sum() + site_meter['export_energy'].sum()
                site_ytd_egrid_lender = site_old_600_ytd['pvsyst_EGrid_lender'].sum()\
                                        + new_table_600.loc[site_id, 'pvsyst_EGrid_lender'].sum()
                site_ytd_egrid_tax = site_old_600_ytd['pvsyst_EGrid_tax'].sum() \
                                        + new_table_600.loc[site_id, 'pvsyst_EGrid_tax'].sum()
                if site_ytd_energy_sum > 0:
                    new_table_600.loc[site_id, 'prod_ytd_lender_a/e'] = (site_ytd_energy_sum / site_ytd_egrid_lender)
                    new_table_600.loc[site_id, 'prod_ytd_tax_a/e'] = (site_ytd_energy_sum / site_ytd_egrid_tax)
                else:
                    new_table_600.loc[site_id, 'prod_ytd_lender_a/e'] = 0
                    new_table_600.loc[site_id, 'prod_ytd_tax_a/e'] = 0
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'prod_ytd_lender_a/e'] = 0
                new_table_600.loc[site_id, 'prod_ytd_tax_a/e'] = 0
            try:
                if not (site_old_600_qtd['meter_net'].sum() + site_meter['net_energy'].sum()) == 0:
                    site_qtd_energy_sum = site_old_600_qtd['meter_net'].sum() + site_meter['net_energy'].sum()
                elif not site_old_600_qtd['meter_export'].sum() + site_meter['export_energy'].sum() == 0:
                    site_qtd_energy_sum = site_old_600_qtd['meter_export'].sum() + site_meter['export_energy'].sum()
                site_qtd_egrid_lender = site_old_600_qtd['pvsyst_EGrid_lender'].sum() \
                                        + new_table_600.loc[site_id, 'pvsyst_EGrid_lender'].sum()
                site_qtd_egrid_tax = site_old_600_qtd['pvsyst_EGrid_tax'].sum() \
                                     + new_table_600.loc[site_id, 'pvsyst_EGrid_tax'].sum()
                if site_qtd_energy_sum > 0:
                    new_table_600.loc[site_id, 'prod_qtd_lender_a/e'] = (site_qtd_energy_sum / site_qtd_egrid_lender)
                    new_table_600.loc[site_id, 'prod_qtd_tax_a/e'] = (site_qtd_energy_sum / site_qtd_egrid_tax)
                else:
                    new_table_600.loc[site_id, 'prod_qtd_lender_a/e'] = 0
                    new_table_600.loc[site_id, 'prod_qtd_tax_a/e'] = 0
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'prod_qtd_lender_a/e'] = 0
                new_table_600.loc[site_id, 'prod_qtd_tax_a/e'] = 0
            try:
                if not site_old_600_hytd['meter_net'].sum() + site_meter['net_energy'].sum() == 0:
                    site_hytd_energy_sum = site_old_600_hytd['meter_net'].sum() + site_meter['net_energy'].sum()
                elif not site_old_600_hytd['meter_export'].sum() + site_meter['export_energy'].sum() == 0:
                    site_hytd_energy_sum = site_old_600_hytd['meter_export'].sum() + site_meter['export_energy'].sum()
                site_hytd_egrid_lender = site_old_600_hytd['pvsyst_EGrid_lender'].sum() \
                                        + new_table_600.loc[site_id, 'pvsyst_EGrid_lender'].sum()
                site_hytd_egrid_tax = site_old_600_hytd['pvsyst_EGrid_tax'].sum() \
                                     + new_table_600.loc[site_id, 'pvsyst_EGrid_tax'].sum()
                if site_hytd_energy_sum > 0:
                    new_table_600.loc[site_id, 'prod_hytd_lender_a/e'] = (site_hytd_energy_sum / site_hytd_egrid_lender)
                    new_table_600.loc[site_id, 'prod_hytd_tax_a/e'] = (site_hytd_energy_sum / site_hytd_egrid_tax)
                else:
                    new_table_600.loc[site_id, 'prod_hytd_lender_a/e'] = 0
                    new_table_600.loc[site_id, 'prod_hytd_tax_a/e'] = 0
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'prod_hytd_lender_a/e'] = 0
                new_table_600.loc[site_id, 'prod_hytd_tax_a/e'] = 0

            # inv
            try:
                new_table_600.loc[site_id, 'inv_export_sum'] = site_inverters['export_energy'].sum()
                new_table_600.loc[site_id, 'inv_count'] = site_inverters['export_energy'].count()
                new_table_600.loc[site_id, 'inv_record_avg_count'] = site_inverters['count'].mean()
                new_table_600.loc[site_id, 'inv_day_count'] = site_inverters['day_count'].mean()
                new_table_600.loc[site_id, 'inv_first_seen'] = site_inverters['first_seen'].min()
                new_table_600.loc[site_id, 'inv_last_seen'] = site_inverters['last_seen'].min()
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'inv_export_sum'] = 0
                new_table_600.loc[site_id, 'inv_count'] = 0
                new_table_600.loc[site_id, 'inv_record_avg_count'] = 0
                new_table_600.loc[site_id, 'inv_day_count'] = 0
                new_table_600.loc[site_id, 'inv_first_seen'] = 0
                new_table_600.loc[site_id, 'inv_last_seen'] = 0
            # ws
            try:
                #print(abs(site_ws['poa_whm2_group_avg'].iloc[0] - site_ws['poa_whm2'].iloc[0])\
                #            / site_ws['poa_whm2'].iloc[0])
                if site_ws['poa_whm2'].mean() > 0:
                    new_table_600.loc[site_id, 'poa_whm2'] = site_ws['poa_whm2'].mean()
                    if site_ws['mm_poa_whm2'].mean() > 0:
                        if site_ws['mm_poa_whm2'].mean() <= 0:
                            new_table_600.loc[site_id, 'poa_whm2'] = site_ws['mm_poa_whm2'].mean()
                            new_table_600.loc[site_id, 'notes'] = 'using_mm_weather, '
                        if abs(site_ws['mm_poa_whm2'].mean() - site_ws['poa_whm2'].mean())\
                                / site_ws['poa_whm2'].mean() >= 0.25:
                            new_table_600.loc[site_id, 'poa_whm2'] = site_ws['mm_poa_whm2'].mean()
                            new_table_600.loc[site_id, 'notes'] = 'using_mm_weather, '
                else:
                    new_table_600.loc[site_id, 'poa_whm2'] = site_ws['mm_poa_whm2'].mean()
                    new_table_600.loc[site_id, 'notes'] = 'using_mm_weather, '
                new_table_600.loc[site_id, 'ghi_whm2'] = site_ws['ghi_whm2'].mean()
                new_table_600.loc[site_id, 'pvsyst_GlobHor'] = site_pvsyst_month['GlobHor'].mean()
                new_table_600.loc[site_id, 'pvsyst_GlobInc'] = site_pvsyst_month['GlobInc'].mean()
                new_table_600.loc[site_id, 'ws_record_count'] = site_ws['count'].mean()
                new_table_600.loc[site_id, 'ws_day_count'] = site_ws['day_count'].mean()
                new_table_600.loc[site_id, 'ws_first_seen'] = str(site_ws['first_seen'].astype(str).iloc[0])
                new_table_600.loc[site_id, 'ws_last_seen'] = str(site_ws['last_seen'].astype(str).iloc[0])
                new_table_600.loc[site_id, 'cloudy_days'] = site_ws['cloudy_days'].mean()
                new_table_600.loc[site_id, 'rainy_days'] = site_ws['rainy_days'].mean()
                new_table_600.loc[site_id, 'snow_days'] = site_ws['snow_days'].mean()
                new_table_600.loc[site_id, 'snowdepth_min'] = site_ws['snowdepth_min'].mean()
                new_table_600.loc[site_id, 'snowdepth_max'] = site_ws['snowdepth_max'].mean()
                new_table_600.loc[site_id, 'snowdepth_mean'] = site_ws['snowdepth_mean'].mean()
                # meteomatics
                new_table_600.loc[site_id, 'meteomatics_poa_whm2'] = site_ws['mm_poa_whm2'].mean()
                new_table_600.loc[site_id, 'meteomatics_ghi_whm2'] = site_ws['mm_ghi_whm2'].mean()
                #actual
                new_table_600.loc[site_id, 'ws_poa_whm2'] = site_ws['poa_whm2'].mean()
                new_table_600.loc[site_id, 'ws_ghi_whm2'] = site_ws['ghi_whm2'].mean()
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'poa_whm2'] = 0
                new_table_600.loc[site_id, 'ghi_whm2'] = 0
                new_table_600.loc[site_id, 'pvsyst_GlobHor'] = 0
                new_table_600.loc[site_id, 'pvsyst_GlobInc'] = 0
                new_table_600.loc[site_id, 'ws_record_count'] = 0
                new_table_600.loc[site_id, 'ws_day_count'] = 0
                new_table_600.loc[site_id, 'meteomatics_poa_whm2'] = 0
                new_table_600.loc[site_id, 'meteomatics_ghi_whm2'] = 0
                new_table_600.loc[site_id, 'ws_poa_whm2'] = 0
                new_table_600.loc[site_id, 'ws_ghi_whm2'] = 0
                new_table_600.loc[site_id, 'cloudy_days'] = 0
                new_table_600.loc[site_id, 'rainy_days'] = 0
                new_table_600.loc[site_id, 'snow_days'] = 0
                new_table_600.loc[site_id, 'snowdepth_min'] = 0
                new_table_600.loc[site_id, 'snowdepth_max'] = 0
                new_table_600.loc[site_id, 'snowdepth_mean'] = 0
            try:
                #variousc poa a/e calcs
                new_table_600.loc[site_id, 'poa_monthly_a/e'] = (new_table_600.loc[site_id, 'poa_whm2'] /
                                                                   site_pvsyst_month['GlobInc'].sum())
                new_table_600.loc[site_id, 'ghi_monthly_a/e'] = (site_ws['ghi_whm2'].mean() /
                                                                   site_pvsyst_month['GlobHor'].sum())
                new_table_600.loc[site_id, 'poa_qtd_a/e'] = ((site_old_600_qtd['poa_whm2'].sum() +
                                                                      site_ws['poa_whm2'].mean()) /
                                                                     site_pvsyst_qtd['GlobInc'].sum())
                new_table_600.loc[site_id, 'ghi_qtd_a/e'] = ((site_old_600_qtd['ghi_whm2'].sum() +
                                                                      site_ws['ghi_whm2'].mean()) /
                                                                     site_pvsyst_qtd['GlobHor'].sum())
                new_table_600.loc[site_id, 'poa_hytd_a/e'] = ((site_old_600_hytd['poa_whm2'].sum() +
                                                                      site_ws['poa_whm2'].mean()) /
                                                                     site_pvsyst_hytd['GlobInc'].sum())
                new_table_600.loc[site_id, 'ghi_hytd_a/e'] = ((site_old_600_hytd['ghi_whm2'].sum() +
                                                                      site_ws['ghi_whm2'].mean()) /
                                                                     site_pvsyst_hytd['GlobHor'].sum())
                new_table_600.loc[site_id, 'poa_ytd_a/e'] = ((site_old_600_ytd['poa_whm2'].sum() +
                                                                      site_ws['poa_whm2'].mean()) /
                                                                     site_pvsyst_ytd['GlobInc'].sum())
                new_table_600.loc[site_id, 'ghi_ytd_a/e'] = ((site_old_600_ytd['ghi_whm2'].sum() +
                                                                      site_ws['ghi_whm2'].mean()) /
                                                                     site_pvsyst_ytd['GlobHor'].sum())
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'poa_monthly_a/e'] = 0
                new_table_600.loc[site_id, 'ghi_monthly_a/e'] = 0
                new_table_600.loc[site_id, 'poa_qtd_a/e'] = 0
                new_table_600.loc[site_id, 'ghi_qtd_a/e'] = 0
                new_table_600.loc[site_id, 'poa_hytd_a/e'] = 0
                new_table_600.loc[site_id, 'ghi_hytd_a/e'] = 0
                new_table_600.loc[site_id, 'poa_ytd_a/e'] = 0
                new_table_600.loc[site_id, 'ghi_ytd_a/e'] = 0
                new_table_600.loc[site_id, 'ws_first_seen'] = 0
                new_table_600.loc[site_id, 'ws_last_seen'] = 0

            try:
                # calc e/a for weather adjustments on poa
                new_table_600.loc[site_id, 'poa_monthly_e/a'] = (site_pvsyst_month['GlobInc'].sum() / new_table_600.loc[site_id, 'poa_whm2'])
                new_table_600.loc[site_id, 'poa_qtd_e/a'] = (site_pvsyst_qtd['GlobInc'].sum() /
                                                             (site_old_600_qtd['poa_whm2'].sum() + site_ws['poa_whm2'].mean()))
                new_table_600.loc[site_id, 'poa_hytd_e/a'] = (site_pvsyst_hytd['GlobInc'].sum() /
                                                              (site_old_600_hytd['poa_whm2'].sum() + site_ws['poa_whm2'].mean()))
                new_table_600.loc[site_id, 'poa_ytd_e/a'] = (site_pvsyst_ytd['GlobInc'].sum() /
                                                             (site_old_600_ytd['poa_whm2'].sum() + site_ws['poa_whm2'].mean()))
            except:
                logging.exception("traceback")
                new_table_600.loc[site_id, 'poa_monthly_e/a'] = 0
                new_table_600.loc[site_id, 'poa_qtd_e/a'] = 0
                new_table_600.loc[site_id, 'poa_hytd_e/a'] = 0
                new_table_600.loc[site_id, 'poa_ytd_e/a'] = 0
                new_table_600.loc[site_id, 'ws_first_seen'] = 0
                new_table_600.loc[site_id, 'ws_last_seen'] = 0
            # weather adjusted prod a/e
            try:
                new_table_600.loc[site_id, 'prod_monthly_lender_wa_a/e'] = new_table_600.loc[site_id, 'poa_monthly_e/a'] * \
                                                                        new_table_600.loc[site_id, 'prod_monthly_lender_a/e']
                new_table_600.loc[site_id, 'prod_monthly_tax_wa_a/e'] = new_table_600.loc[site_id, 'poa_monthly_e/a'] * \
                                                                     new_table_600.loc[site_id, 'prod_monthly_tax_a/e']
                new_table_600.loc[site_id, 'prod_ytd_lender_wa_a/e'] = new_table_600.loc[site_id, 'poa_ytd_e/a'] * \
                                                                    new_table_600.loc[site_id, 'prod_ytd_lender_a/e']
                new_table_600.loc[site_id, 'prod_ytd_tax_wa_a/e'] = new_table_600.loc[site_id, 'poa_ytd_e/a'] * \
                                                                 new_table_600.loc[site_id, 'prod_ytd_tax_a/e']
                new_table_600.loc[site_id, 'prod_qtd_lender_wa_a/e'] = new_table_600.loc[site_id, 'poa_qtd_e/a'] * \
                                                                    new_table_600.loc[site_id, 'prod_qtd_lender_a/e']
                new_table_600.loc[site_id, 'prod_qtd_tax_wa_a/e'] = new_table_600.loc[site_id, 'poa_qtd_e/a'] * \
                                                                 new_table_600.loc[site_id, 'prod_qtd_tax_a/e']
                new_table_600.loc[site_id, 'prod_hytd_lender_wa_a/e'] = new_table_600.loc[site_id, 'poa_hytd_e/a'] * \
                                                                     new_table_600.loc[site_id, 'prod_hytd_lender_a/e']
                new_table_600.loc[site_id, 'prod_hytd_tax_wa_a/e'] = new_table_600.loc[site_id, 'poa_hytd_e/a'] * \
                                                                  new_table_600.loc[site_id, 'prod_hytd_tax_a/e']
            except:
                new_table_600.loc[site_id, 'prod_monthly_lender_wa_a/e'] = 0
                new_table_600.loc[site_id, 'prod_monthly_tax_wa_a/e'] = 0
                new_table_600.loc[site_id, 'prod_ytd_lender_wa_a/e'] = 0
                new_table_600.loc[site_id, 'prod_ytd_tax_wa_a/e'] = 0
                new_table_600.loc[site_id, 'prod_qtd_lender_wa_a/e'] = 0
                new_table_600.loc[site_id, 'prod_qtd_tax_wa_a/e'] = 0
                new_table_600.loc[site_id, 'prod_hytd_lender_wa_a/e'] = 0
                new_table_600.loc[site_id, 'prod_hytd_tax_wa_a/e'] = 0



            # make final column, mon_prod, which will choose meter_net, meter_export or inv_export in that order
            if new_table_600.loc[site_id, 'meter_net'] > 0:
                new_table_600.loc[site_id, 'month_prod'] = new_table_600.loc[site_id, 'meter_net']
            elif new_table_600.loc[site_id, 'meter_export'] > 0:
                new_table_600.loc[site_id, 'month_prod'] = new_table_600.loc[site_id, 'meter_export']
            elif new_table_600.loc[site_id, 'inv_export_sum'] > 0:
                new_table_600.loc[site_id, 'reporting'] = 'y'
                new_table_600.loc[site_id, 'notes'] = new_table_600.loc[site_id, 'notes'] + 'using sum of inv data, '
                new_table_600.loc[site_id, 'month_prod'] = new_table_600.loc[site_id, 'inv_export_sum']
            else:
                new_table_600.loc[site_id, 'month_prod'] = 0

            #uptime
            try:
                new_table_600 = uptime(new_table_600, days_ago, site, site_ws, site_inverters, site_meter, all_devices)
            except:
                new_table_600.loc[site['id'], 'uptime'] = "n/a"
                logging.exception("traceback")
                print('uptime failed for ', site['site_name'])
        new_table_600 = new_table_600.replace(np.inf, 0)
        new_table_600.to_csv("S:/CHI1/Company Data/O&M/Diamond_Williams/Reporting/table_600.csv", index=True)
        this_month_old_600 = old_table_600[old_table_600['month'] == str(datetime.strftime(datetime.today() - timedelta(days=days_ago), '%Y-%m'))]
        print("Updating ", len(new_table_600), " Records")
        print(new_table_600)
        for idx, record in new_table_600.iterrows():
            print(idx)
            print(record)
            if len(this_month_old_600[this_month_old_600['site_id'] == record['site_id']]) > 0:
                update_records(record)
            else:
                add_new_records(record)


if __name__ == '__main__':
    main()

1
2020-05
